The code below does not work any more due to Instagram changing its policies on accessing its endpoints.

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests
import urllib.request
import urllib.parse
import urllib.error
import ssl
import json
import pandas as pd
import numpy as np
from datetime import datetime
import time
import tqdm
import traceback
import requests
import random
import pickle
import urllib.request as req

In [ ]:
hashtagList = ['ejuice', 'eliquid'];

In [ ]:
class Insta_Image_Links_Scraper:

    def getlinks(self, hashtag, url, count):

        html = urllib.request.urlopen(url, context=self.ctx).read()

        data = json.loads(html)
        vals = []
        for node in data['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
            post = node['node']
            temp = {}
            try:
                temp['linkCode'] = post['shortcode']
                temp['isVideo'] = post['is_video']
                temp['postType'] = post['__typename']
                temp['postId'] = post['id']
                
                if post['edge_media_to_caption']['edges']:
                    temp['postCaption'] = post['edge_media_to_caption']['edges'][0]['node']['text']
                else:
                    temp['postCaption'] = ''
                    
                temp['imageLink'] = post['display_url']                
                temp['postLink'] = "https://www.instagram.com/p/" + temp['linkCode']
                temp['commentCount'] = post['edge_media_to_comment']['count']
                temp['timeStamp'] = post['taken_at_timestamp']
                temp['numLikes'] = post['edge_media_preview_like']['count']
                temp['caption'] = post['accessibility_caption']
                if temp['postType'] == 'GraphSidecar' or temp['isVideo']:
                    html_temp =  urllib.request.urlopen(temp['postLink'] + '/?__a=1', context=self.ctx).read()
                    pageData_temp = json.loads(html_temp)
                    pageData_temp = pageData_temp['graphql']['shortcode_media']
                    
                    if temp['isVideo']:
                        temp['videoURL'] = pageData_temp['video_url']
                        temp['videoViewCount'] = pageData_temp['video_view_count']
                    else:
                        edgeLinks = []
                        for node in pageData_temp['edge_sidecar_to_children']['edges']:
                            edgeLinks.append(node['node']['display_url'])
                        temp['imgURLList'] = edgeLinks
                vals.append(temp)
                count+=1
            except Exception as e:
                print(e)
                print("link: " + shortcode)
            
        end_cursor = data['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']

        return vals, end_cursor, count 

    def main(self, hashtag, data, max_count = 10000):
        self.ctx = ssl.create_default_context()
        self.ctx.check_hostname = False
        self.ctx.verify_mode = ssl.CERT_NONE
        print('https://www.instagram.com/explore/tags/'
                      + hashtag + '/?__a=1')
        html = urllib.request.urlopen('https://www.instagram.com/explore/tags/'
                      + hashtag + '/?__a=1', context=self.ctx).read()
        page = json.loads(html)
        if(max_count > page['graphql']['hashtag']['edge_hashtag_to_media']['count']):
            max_count = page['graphql']['hashtag']['edge_hashtag_to_media']['count']
            print("Number of posts: " + str(max_count))
        
        print("Current Hashtag: #"+hashtag)
        count = 0
        vals, end_cursor, count = self.getlinks(hashtag,
                      'https://www.instagram.com/explore/tags/'
                      + hashtag + '/?__a=1', count)
        data.extend(vals)
        print(count)
        while count < max_count:
#                 print("New link: " + 'https://www.instagram.com/explore/tags/' 
#                       + hashtag + '/?__a=1&max_id=' + end_cursor)
            try:
                if end_cursor == None:
                    break;
                vals, end_cursor, count = self.getlinks(hashtag,
                          'https://www.instagram.com/explore/tags/'
                          + hashtag + '/?__a=1&max_id=' + end_cursor, count)
                data.extend(vals)
                print(count) 
            except Exception as e:
                print("Error: " + str(e))              
                time.sleep(random.uniform(1, 3))

In [ ]:
obj = Insta_Image_Links_Scraper()

In [ ]:
data = []

In [ ]:
obj.main(hashtagList[0], data)
print(len(data))
file = open('data_ejuice.pickle', 'wb')
pickle.dump(data, file)
file.close()